In [59]:

import time
import requests
import os
import datetime
import yaml

from rich import print
from dotenv import load_dotenv


load_dotenv('./.env')
nai_api_key = os.getenv('NAI_API_KEY')
# print(nai_api_key)
hf_api_key = os.getenv('HF_TOKEN')

nai_url= 'https://ai.nutanix.com/api/v1/chat/completions'

nai_api_key = nai_api_key.strip()

def send_inference_request(system_prompt, user_prompt):
    inference_input = {
        "model": "vllm-llama-3-1",
        "messages": [
            {
                "role": "system",
                "content": system_prompt
            },
            {
                "role": "user",
                "content": user_prompt
            }
        ],
        "max_tokens": 1024,
        "stream": False
    }

    response = requests.post(nai_url, 
                            headers={
                                    "Authorization": f"Bearer {nai_api_key}", 
                                    "accept": "application/json",
                                    "Content-Type": "application/json"
                                }, 
                            json=inference_input, 
                            verify=False)
    return response.json()


In [47]:
source_code_file = 'codegen-observation/hritik/cluster.go/cluster.go'
source_code_manual_unit_test_file ='codegen-observation/hritik/cluster.go/cluster_test.go'
source_code_codegen_unit_test_file = 'codegen-observation/hritik/cluster.go/cluster_test_codegen.go'

In [49]:
with open('Prompts/prompt_01.yaml', 'r') as file:
    prompts = yaml.safe_load(file)

with open(source_code_file, 'r') as source_file:
    go_source_code = source_file.read()

with open(source_code_manual_unit_test_file, 'r') as test_file:
    go_unit_test_code = test_file.read()

with open(source_code_codegen_unit_test_file, 'r') as test_file:
    go_unit_test_code = test_file.read()
    
# print(prompts)
system_prompt = prompts.get('system_prommpt','')

user_prompt = prompts.get('user_prompt', '')
user_prompt = user_prompt.replace('<GO_SOURCE_CODE>', go_source_code).replace('<GO_MANUAL_UNIT_TEST_CODE>', source_code_manual_unit_test_file).replace('<GO_COPILOT_UNIT_TEST_CODE>',source_code_codegen_unit_test_file)

# print(user_prompt)

In [ ]:
response_from_model = send_inference_request(system_prompt=system_prompt, user_prompt=user_prompt)["choices"][0]["message"]["content"]

with open('./Responses/response_01.txt', 'w') as file:
    file.write(response_from_model)


## Feedback prompt to the model


## Feedback from prompt 1 to 2


In [45]:
with open('./Prompts/Prompt_02.yaml','r') as file:
    feedback_prompt = yaml.safe_load(file)
    

with open('./Responses/response_01.yaml','r') as file:
    response_from_prompt_1 = yaml.safe_load(file)
    
feedback_system_prompt = feedback_prompt.get('system_prompt','')

feedback_user_prompt = feedback_prompt.get('user_prompt','').replace('<CHANGES_NEEDED>',response_from_prompt_1).replace('<GO_UNIT_TEST_CODE>', go_unit_test_code)

# print(feedback_user_prompt)

In [46]:
inference_input = {
    "model": "vllm-llama-3-1",
    "messages": [
        {
            "role": "system",
            "content": feedback_system_prompt
        },
        {
            "role": "user",
            "content": feedback_user_prompt
        }
    ],
    "max_tokens": 256,
    "stream": False
}

response = requests.post(nai_url, 
                         headers={
                                "Authorization": f"Bearer {nai_api_key}", 
                                "accept": "application/json",
                                "Content-Type": "application/json"
                            }, 
                         json=inference_input, 
                         verify=False)


feeback_response_from_model = response.json()["choices"][0]["message"]["content"]

with open('./Responses/response_02.yaml','w') as file:
    yaml.dump(feeback_response_from_model, file)

/Users/hritik.raj/codegen-nai/.venv/lib/python3.12/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ai.nutanix.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
